# Introdução ao Spark Streaming

### Spark Streaming
A proposta do Spark Stream é analisar dados em tempo real, e não esperar horas para fazer a análise e processamento. "*A vida não acontece em batches*".<br/>
Streaming de dados não é apenas para projetos altamente especializados. Computação baseada em Streaming está se tornando a regra para empresas orientadas a dados.<br/>
Uma das principais fontes de dados contínuos são os sensores, da internet das coisas.
Existem quatro areas principais que o Spark Streaming vem sendo utilizado:
* Streaming ETL;
* Detecção de anomalias;
* Enriquecimento de dados;
* Sessões complexas e aprendizado contínuo.

*Uma importante vantagem de usar o Spark para Big Data Analytics é a possibilidade de combinar processamento em batch e processamento de streaming em um único sistema*.<br/.
* **Batch**: Você inicia o processamento de um arquivo ou dataset finito, o spark processa as tarefas configuradas e conclui o trabalho.
* **Streaming**: Você processa um stream de dados contínuos; a execução não pára até que haja algum erro ou você termine a aplicação manualmente.

### DStreams: Discretized Streams
Assim como os RDDs são a base do Apache Spark, os DStreams são a base do Apache Spark Streaming.<br/>
O dstream é uma sequência de dados que são coletados ao longo do tempo. Internamente, um dstream é representado por uma sequência de RDDs coletados em cada intervalo de tempo. Pode ser criado por diversas fontes, como Kafka, Flume, Twitter, etc..<br/>
Uma vez que são criados, os dstreams oferecem dois tipos e operações:
* **Transformações**: Geram um novo dstream;
* **Ações (operações de output)**: Gravam os dados em um sistema de armazenamento ou outra fonte externa.
Os DStreams oferecem muitas das operações que podem ser realizadas com os RDDs, mais operações relacionadas ao tempo, como sliding windows.

<img src="assets/figura02.png"/>

**O que pode ser feito com DStreams:**
* Map;
* FlatMap;
* Filter;
* ReduceByKey;
* Join;
* Window;
* Manter o controle de estado dos dados (Stateful Data)

**Windowing**
Computação em uma janela de tempo. A cada janela de tempo, um RDD é criado no DStream, podemos querer ver o que acontece em um determinado intervalo de tempo.
* Window length: Duração da window;
* Sliding interval: Intervalo entre as windows.

Exemplo de uso:
> ssc = StreamingContext(sc, INTERVALO_BATCH);<br/>
window(windowDuration: Duration, slideDuration: Duration): DStream[T]<br/>

Windowing permite computar os resultados ao longo de períodos de tempo maiores que o batch interval.

São três os intervalos de tempo que devemos considerar ao trabalhar com streaming:
* Batch interval: Frequência com que os dados são capturados em um DStream;
* Frequência com que uma window é aplicada;
* Intervalo de tepo capturado para computação e geração de resultados;

**Principais mecanismos de tolerância a falhas:**
* Todos os dados são replicados para no mínimo 2 worker nodes;
* *ssc.checkpoint()*: Um diretório de checkpoint pode ser usado para armazenar o estado do streaming de dados, no caso em que é necessário reiniciar o streaming.
* *Falha no Receiver*: Alguns receivers são melhores que outros. Receiver como Twitter, Kafka e Flume não permitem recuperação de dados. Se o receiver falha, os dados do streaming são perdidos. Outros garantem a recuperação dos dados em caso de falhas: HDFS, Directly-consumed Kafka, Pull-based Flume.
* *Falha no Driver Context*: Embora os dados sejam replicados para os worker nodes, o DriverContext é executado no node master e este pode ser um ponto único de falha. Podemos usar o checkpoint() para recuperar dados em caso de falhas e usamos a função streamingContext.getOrCreate() para continuar o processamento de onde ele foi interrompido em caso de falha.
Em caso de falha no script sendo executado no DriverContext, podemos reiniciar automaticamente o processo de streaming, usando o Zookeeper (no modo supervise). O zookeeper é um cluster manager usado pelo spark.

**Integração com outros sistemas: Kafka, Flume e Kinesis.**<br/>
É um sistema para gerenciamento de fluxos de dados em tempo real. O Linkedin é o maior ambiente Kafka atualmente. Foi desenvolvido com o propósito de servir como repositório de dados gerados em tempo real. Foi desenvolvido com dois objetivos em mente:
* Transportar dados entre diversos sistemas de dados;
* Enriquecer a análise de dados.

Kafka com spark: Instalar o pacote **spark-streaming-kafka**

**Apache Flume**<br/>
É parecido com o Kafka, mas é específico para logs em servidores, e pode ser muito útil para o processo de análise de dados.
Instalação:
* *Spark-streaming-flume*: Push-based;
* *Spark-streaming-flume-sink*: Pull-Based.

**Amazon Kinesis**<br/>
É similar ao kafka, mas pode ser integrado direto com a nuvem da amazon.
* *Spark-streaming-kinesis-asl*: Requer a licença da Amazon.



### Para acompanhar a execução dos jobs basta acessar o seguinte endereço:
Acesse http://localhost:4040 sempre que quiser acompanhar a execução dos jobs

### Simulando um Streaming de Dados TCP/IP na porta 9898 do seu computador local
Windows: http://mobaxterm.mobatek.net/ . Clique em Start Local Terminal e depois: nc -lk 9898 <br/>
Mac/Linux: nc -lk 9898



In [2]:
# Importando o StreamingContext
from pyspark.streaming import StreamingContext

In [3]:
# Criando um StreamingContext com intervalo batch de 1 segundo
ssc = StreamingContext(sc, 1)

In [4]:
# Criando um DStream que vai conectar na porta 9898 da sua máquina local
linhas = ssc.socketTextStream("localhost", 9898)

In [5]:
type(linhas)

pyspark.streaming.dstream.DStream

In [6]:
# Divide cada linha em palavras
palavras = linhas.flatMap(lambda line: line.split(" "))

In [7]:
type(palavras)

pyspark.streaming.dstream.TransformedDStream

In [8]:
# Conta cada palavra em cada batch
pares = palavras.map(lambda palavra: (palavra, 1))
contaPalavras = pares.reduceByKey(lambda x, y: x + y)

In [9]:
# Imprime os 10 primeiros elementos de cada rDD gerado no DStream
contaPalavras.pprint()

In [ ]:
ssc.start()             # Inicia a coleta e processamento do stream de dados
ssc.awaitTermination()  # Aguarda a computação ser finalizada

-------------------------------------------
Time: 2022-05-24 13:51:18
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:19
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:20
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:21
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:22
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:23
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:51:24
-------------------------------------------
('hi', 1)

-------------------------------------------
Time: 2022-05-24 13:51:25
-------------------------------------------
('henrique', 1)

-------------------------------------------
Time: 2022

-------------------------------------------
Time: 2022-05-24 13:52:29
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:30
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:31
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:32
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:33
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:34
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:35
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:36
-------------------------------------------

-------------------------------------------
Time: 2022-05-24 13:52:37
----------

In [ ]:
ssc.stop()